Модели типа logistic с фичами по товарам и пользователям с кросс валидацией

In [ ]:
pip install lightfm

     |████████████████████████████████| 307kB 3.4MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707618 sha256=887842bdc652a51ac797b46dc672b71c9b5685835f5c43470418cf382b8fcc37
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

DIR = '/content/drive/My Drive/Colab Notebooks/Data_Sceince/Module_5/recommendationsv4/'
DIR_TEST = '/content/drive/My Drive/Colab Notebooks/Data_Sceince/Module_5/TEST/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm


In [ ]:
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

from lightfm.data import Dataset

In [ ]:
#Читаем файлы
data = pd.read_csv(DIR + 'train.csv',low_memory=False )
test = pd.read_csv(DIR +'test.csv',low_memory=False )
submission = pd.read_csv(DIR +'sample_submission.csv')

In [ ]:
fichi = pd.read_csv(DIR + 'fi4i30T.csv',low_memory=False) #, nrows = 5 )

In [ ]:
fichi.describe()

In [ ]:
fichi.head()

,itemid,userid,overall,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,37138,102179,5.0,0,-0.481626,-0.146912,-0.461433,0.627254,0.490984,0.068531,-0.175226,-0.101846,-0.392405,-0.067195,-0.881019,-0.086871,-0.111445,0.019312,-0.323024,0.299162,0.178909,0.363466,0.054027,0.252262,0.131708,-0.467254,0.025174,-0.539695,-0.438329,0.651743,0.215715,0.094889,0.171043,0.420939
1,17322,3625,5.0,1,-0.487150,-0.490408,-0.863708,1.268813,0.201121,-0.341024,-0.014472,0.182113,-0.661514,0.008635,-0.278071,0.004864,0.820423,0.140630,-0.317460,-0.323987,0.563939,0.943316,0.593163,0.602663,0.360174,0.139982,-0.408589,0.036960,0.035105,-0.148636,0.650877,-0.138781,0.217508,0.681503
2,5600,39495,5.0,2,-0.030458,-0.482874,0.305205,0.531545,0.027049,0.200568,0.163929,0.374942,-0.366942,0.171773,0.039992,-0.521756,0.514570,-0.051424,-0.705174,-0.092675,0.178857,0.180742,0.430242,-0.125441,0.086506,0.553647,-0.293880,0.111545,0.014612,-0.491548,0.151792,0.182814,-0.080710,0.355633
3,30249,72854,5.0,3,-0.265692,-0.518816,-0.034628,0.370477,0.138180,-0.112502,0.164425,-0.046640,-0.102842,-0.267234,-0.751457,0.183983,-0.119559,0.356675,-0.414000,0.159884,0.096142,0.354406,0.451528,-0.189015,0.048574,-0.288945,-0.032130,-0.576467,-0.210558,0.164137,0.283811,-0.013973,0.111765,-0.013857
4,4349,74859,5.0,4,0.260096,0.089920,0.311937,0.956379,0.644498,0.331261,-0.254293,0.182654,-0.053495,0.599993,0.234138,-0.523378,1.030362,-0.525004,-0.100578,1.263725,0.399474,1.009355,0.203495,0.404312,-0.248540,0.814974,0.719038,-1.018465,-0.174331,0.565271,0.515446,-0.446922,-0.419972,0.017191


In [ ]:
fichi = fichi.rename(columns={'Unnamed: 0':'id'},)
fichi = pd.concat([data[['userid']],fichi], axis =1, copy = 'False')

In [ ]:
fichi.head()

,userid,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,102179,0,-0.481626,-0.146912,-0.461433,0.627254,0.490984,0.068531,-0.175226,-0.101846,-0.392405,-0.067195,-0.881019,-0.086871,-0.111445,0.019312,-0.323024,0.299162,0.178909,0.363466,0.054027,0.252262,0.131708,-0.467254,0.025174,-0.539695,-0.438329,0.651743,0.215715,0.094889,0.171043,0.420939
1,3625,1,-0.487150,-0.490408,-0.863708,1.268813,0.201121,-0.341024,-0.014472,0.182113,-0.661514,0.008635,-0.278071,0.004864,0.820423,0.140630,-0.317460,-0.323987,0.563939,0.943316,0.593163,0.602663,0.360174,0.139982,-0.408589,0.036960,0.035105,-0.148636,0.650877,-0.138781,0.217508,0.681503
2,39495,2,-0.030458,-0.482874,0.305205,0.531545,0.027049,0.200568,0.163929,0.374942,-0.366942,0.171773,0.039992,-0.521756,0.514570,-0.051424,-0.705174,-0.092675,0.178857,0.180742,0.430242,-0.125441,0.086506,0.553647,-0.293880,0.111545,0.014612,-0.491548,0.151792,0.182814,-0.080710,0.355633
3,72854,3,-0.265692,-0.518816,-0.034628,0.370477,0.138180,-0.112502,0.164425,-0.046640,-0.102842,-0.267234,-0.751457,0.183983,-0.119559,0.356675,-0.414000,0.159884,0.096142,0.354406,0.451528,-0.189015,0.048574,-0.288945,-0.032130,-0.576467,-0.210558,0.164137,0.283811,-0.013973,0.111765,-0.013857
4,74859,4,0.260096,0.089920,0.311937,0.956379,0.644498,0.331261,-0.254293,0.182654,-0.053495,0.599993,0.234138,-0.523378,1.030362,-0.525004,-0.100578,1.263725,0.399474,1.009355,0.203495,0.404312,-0.248540,0.814974,0.719038,-1.018465,-0.174331,0.565271,0.515446,-0.446922,-0.419972,0.017191


In [ ]:
data.head()

,overall,verified,reviewTime,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,userid,itemid,rating
139377,5.0,False,"08 9, 2000",B00004S1C5,Stephanie Manley,This are so much easier to use than the Wilson...,Very easy to use,965779200,9,NaN,NaN,142,4,1.0
827509,5.0,False,"11 30, 2002",B00004W4VD,Shari M. Lowery,The Nesco American Harvest Jerky Works has mad...,Outstanding Product...,1038614400,11,NaN,NaN,156,5,1.0
444617,5.0,False,"12 5, 2003",B0000DBN2F,Rebecca of Amazon,Blend a little apple cider mix into your Tazo ...,Spice It Up for the Holidays,1070582400,NaN,NaN,NaN,1874,83,1.0
551053,5.0,False,"12 14, 2003",B000R4LP6M,Westley,Tapatio salsa picante is one of my favorite ho...,Fantastic Smoky Flavor,1071360000,2,NaN,NaN,15372,4326,1.0
506098,5.0,False,"03 26, 2004",B000PILJ5C,Jess,I have suffered from horrible periods since I ...,It does what it promises and has a refreshing ...,1080259200,15,NaN,NaN,73762,4095,1.0


In [ ]:
# Определяем выбросы, вероятно спам
def get_spam(field):
  med = data[field].value_counts().describe()['50%']
  std = data[field].value_counts().describe()['std']
  counts = pd.DataFrame(data[field].value_counts())
  spam = list(counts[counts[field] > med + 3 * std].index)
  return spam

In [ ]:
item_spam = get_spam('itemid')
user_spam = get_spam('userid')
data = data[~((data['itemid'].isin(item_spam)) & (data['userid'].isin(user_spam)))]

In [ ]:
# Упорядочим наши данные по верифицированности ответа и времени
data.sort_values(by = ['verified', 'unixReviewTime'], ascending= True, inplace = True)

In [ ]:
#Удалим дубли, оставим только по одной паре  пользователь товар
#оставим последний отзыв,считая, что  могут меняться вкусы и качество товаров
data.drop_duplicates(subset =['userid','unixReviewTime','itemid'], keep = 'last',inplace = True)


In [ ]:
len(data), len(test),  len(test)/len(data)

(806942, 285965, 0.35438110793588634)

In [ ]:
data = data.reset_index()

In [ ]:
fichi = fichi[fichi['id'].isin(data['index'])]
fichi_user = fichi.groupby('userid').mean()

fichi_user.drop(columns=['id'], inplace=True )
fichi_user

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
userid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.414023,0.573671,-0.466751,-0.486914,0.176791,0.650737,-0.299078,0.055431,-0.106625,-0.036879,-1.102718,-0.257848,-0.912889,0.378128,0.463470,0.063092,-0.416800,-0.488641,0.109133,-0.477623,0.368227,1.281831,-0.851987,1.464920,0.906994,-0.061233,-0.212764,-0.371089,-0.086386,0.669767
1,-0.473889,-0.131186,0.104379,0.110335,-0.297053,0.073472,-0.301158,0.043840,0.099704,-0.127024,-0.342371,-0.192074,-0.111711,0.127679,-0.438065,0.259409,-0.061917,-0.468077,-0.183858,-0.244458,-0.079782,-0.342786,-0.636168,-0.015975,0.343880,-0.239369,-0.359413,-0.213119,0.152553,0.554046
2,-0.083034,0.034994,0.262354,0.665764,-0.237267,-0.247594,0.200038,0.520357,-0.295003,0.072914,-0.094682,-0.410981,0.431512,-0.203372,-0.658337,0.181861,0.489063,0.334488,0.491160,-0.156633,-0.090893,0.023826,-0.151006,-0.188061,0.181420,-0.381782,0.145217,-0.013698,-0.374472,0.526813
3,-0.207925,-0.154537,0.200064,0.632794,-0.103974,0.029883,-0.093777,0.225340,0.050834,-0.044944,-0.098848,-0.085857,0.229972,-0.018683,-0.269769,0.191831,0.279944,0.008246,0.113414,-0.235464,-0.147571,0.407872,-0.303494,0.032493,0.312603,-0.450149,-0.042108,-0.231564,-0.040502,0.450197
4,-0.562584,-0.298769,0.687209,0.027878,-0.229159,-0.037652,-0.367445,0.753560,-0.180722,-0.370081,0.245327,-0.564209,0.224255,0.437975,-0.457192,0.272566,0.174541,-0.424752,0.841026,0.287557,0.357524,-0.372782,-0.283749,0.761596,-0.098853,0.152141,-0.215025,-0.415195,-0.082853,0.344746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127491,-0.171731,-0.292508,-0.084355,0.262250,-0.360784,0.328295,-0.345818,-0.068277,0.164417,0.455658,-0.304206,0.086377,0.322038,0.222412,-0.377792,0.357930,-0.040731,0.408787,-0.115025,0.082772,-0.424482,0.180980,-0.135821,-0.602326,0.043239,-0.367385,-0.061285,-0.282298,0.308017,0.732400
127492,-0.117186,-0.032770,0.206553,0.921429,0.090460,-0.008201,-0.171495,0.287463,-0.523292,0.724387,-0.273232,-0.514711,0.638894,-0.311819,-0.670167,0.581010,0.191574,0.163404,0.160443,-0.207914,-0.163068,0.171015,-0.093331,-0.339307,-0.339726,-0.387873,0.008461,-0.107245,-0.618327,1.007551
127493,-0.293894,-0.303011,-0.067055,0.279993,-0.137539,0.283358,-0.159617,-0.271246,-0.096701,0.414746,-0.505187,0.017551,0.151541,-0.458079,-0.141782,0.367425,0.035769,0.846413,-0.499267,-0.077148,-0.140418,0.231476,0.734651,-0.780837,-0.758331,-0.045737,0.151879,0.273747,-0.255779,0.751151


In [ ]:
%%time
#Разделим всех пользователй на группы, чтобы потом сделать фичи по пользователям
from sklearn.cluster import KMeans

# Создаём KMeans кластеризатор 
kmeans = KMeans(n_clusters=100)

# Обучаем кластеризатор на подготовленных данных
kmeans.fit(fichi_user.to_numpy())

# Получаем предсказанные кластеры
y_pred = kmeans.labels_.astype(np.int)

CPU times: user 10min 24s, sys: 4min 38s, total: 15min 3s
Wall time: 9min 34s


In [ ]:
#Объединим пользователей и их номер кластера в один датафрейм, подготовим данные для передачи в модель
fichi_user_indx = pd.concat([pd.DataFrame(fichi_user.reset_index()['userid']), pd.DataFrame(y_pred, columns =['label'])], axis =1)
fichi_user_indx['userid'] = fichi_user_indx['userid'].astype('int')
fichi_user_indx['user_features'] = fichi_user_indx.apply(lambda x:(int(x.userid), [int(x.label)]), axis =1)
fichi_user_indx

,userid,label
0,0,22
1,1,44
2,2,2
3,3,80
4,4,86
...,...,...
127443,127491,61
127444,127492,2
127445,127493,26
127446,127494,82


In [ ]:
# Проверим , что между  asin и itemid однозначное соответствие, что данные корректны
data_all = pd.concat([data, test])
data_all_test = pd.DataFrame(data_all.groupby(['asin']).itemid.max() - data_all.groupby(['asin']).itemid.min())
data_all_test.itemid.value_counts()
# должен быть ноль по всей длине датафрейма

0    41320
Name: itemid, dtype: int64

In [ ]:
#Словарь соответствия asin и itemid
mapper = dict(zip(data_all.asin, data_all.itemid))
len(mapper)

41320

In [ ]:
import json
def get_by_key(field, dict_from, dict_to):
   if field in dict_from.keys():
       dict_to[mapper.get(dict_from.get('asin'))] = dict_from.get(field)    

with open(DIR + 'meta_Grocery_and_Gourmet_Food_small.json') as file:
     d_main_category = {}
     d_brand = {}
     d_category = {}
     for line in file.readlines(): 
         lines = json.loads(line) 
         get_by_key('brand', lines, d_brand)
         get_by_key('main_cat', lines, d_main_category)
         get_by_key('category', lines, d_category)


In [ ]:
set_itemid = set(data['itemid']) | set(test['itemid']) 
#Подготовим словари с фичами для модели, обеспечим уникальность значения
d_brand_ml =  {x:'brand:' + y for x,y in d_brand.items()}
d_main_category_ml = {x:'main_cat:' + y for x,y in d_main_category.items()}
#Поиграем самой категорией, возьмем самый нижний уровень
d_last_category_ml = {x:'last_cat:' + y[-1] for x,y in d_category.items()}

In [ ]:
#Разбиваем наш датасет на обучающую и тестовую выборки
train_data, test_data = train_test_split(data,random_state=32, shuffle=True) #, test_size = 0.25)

In [ ]:
#Формируем  mapping для всех пользователей и товаров(train + test)

dataset = Dataset() #item_identity_features=False)
dataset.fit((x for x in pd.concat([data, test])['userid']),
            (x for x in pd.concat([data, test])['itemid']))

In [ ]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 127496, num_items 41320.


In [ ]:
# Добавляем фичи по товарам в dataset
dataset.fit_partial(items=(x for x in d_last_category_ml.keys()),
                    item_features=(x for x in d_last_category_ml.values()))

dataset.fit_partial(items=(x for x in d_main_category_ml.keys()),
                    item_features=(x for x in d_main_category_ml.values()))

dataset.fit_partial(items=(x for x in d_brand_ml.keys()),
                    item_features=(x for x in d_brand_ml.values()))

In [ ]:
# Добавляем фичи по пользователям в dataset
dataset.fit_partial(users=fichi_user_indx['userid'],
                          user_features=fichi_user_indx['label'])

In [ ]:
#Готовим основную матрицу
(interactions, weights) = dataset.build_interactions(((x,y,z) 
                for x,y, z in zip(train_data.userid.values,train_data.itemid.values, train_data.rating.values)))
              #  for x,y, z in zip(data.userid.values,data.itemid.values, data.rating.values)))

print(repr(interactions))

<127496x41320 sparse matrix of type '<class 'numpy.int32'>'
	with 806942 stored elements in COOrdinate format>


In [ ]:
# Добавляем фичи по товарам 
def get_item_features(itemid):
   def add_from_dict(dict_):
      if dict_.get(itemid):
         list_feat.append(dict_.get(itemid)) 

   list_feat =[]
   add_from_dict(d_brand_ml)
   add_from_dict(d_last_category_ml)
   add_from_dict(d_main_category_ml)
   return list_feat

item_features = dataset.build_item_features(((x, get_item_features(x))
                                                       for x in set_itemid ))
print(repr(item_features))

<41320x51173 sparse matrix of type '<class 'numpy.float32'>'
	with 164772 stored elements in Compressed Sparse Row format>


In [ ]:
# Добавляем фичи по пользователям
user_features = dataset.build_user_features((x for x in list(fichi_user_indx['user_features'])))

In [ ]:
# Вытаскиваем словари для преобразований
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [ ]:
from lightfm import LightFM
NUM_THREADS = 8 #число потоков
RANDOM_SEED = 32
#Гиперпараметры
#NUM_EPOCHS, LEARNING_RATE, NUM_COMPONENTS, ALPHA =  [108,0.9307615113531499,156,1.117646374973343e-06]
#params = [15,0.1,120,0]  #стартовые параметры
#params = [108,0.9307615113531499,156,1.117646374973343e-06] # без фич
params =  [54, 0.6695553384725956, 90, 4.527672167401107e-06] #с фичами

In [ ]:
def prediction_model(mtype, h_params, mat_weights):
    num_epochs, learning_rate, num_components, alpha = h_params
    model = LightFM(learning_rate = learning_rate, 
                          loss = mtype, 
                          no_components = num_components,
                          random_state = RANDOM_SEED,
                          user_alpha = alpha,
                          item_alpha = alpha)
    model = model.fit(mat_weights, epochs=num_epochs, num_threads=NUM_THREADS
                              , item_features=item_features
                      , user_features=user_features
                              ) 
    
    return(model)


N_FOLDS = 5
sample_submissions = pd.DataFrame(0,columns=["sub_1"], index=submission.index) # куда пишем предикты по каждой модели
score_ls = []
X = data[['userid','itemid']]
y = data.rating
#splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))

splits = list(StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))


for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=N_FOLDS,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # Строим разреженную матрицу 
#Готовим основную матрицу
    (interactions, weights_train) = dataset.build_interactions(((x,y,z) 
                for x,y, z in zip(X_train.userid.values, X_train.itemid.values, y_train)))
    # model for this fold
    model_light = prediction_model('logistic', params, weights_train)

    # score model on test
    test_predict = model_light.predict(X_test.userid.apply(lambda x: user_id_map[x]).values,
                              X_test.itemid.apply(lambda x: item_id_map[x]).values 
                             ,item_features=item_features
                             , user_features=user_features
                             )

    test_score = sklearn.metrics.roc_auc_score(y_test,test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {test_score:0.4f}")
    # submissions
    # submissions
    sample_submissions[f'sub_{idx+1}'] = model_light.predict(test.userid.apply(lambda x: user_id_map[x]).values,
                                                             test.itemid.apply(lambda x: item_id_map[x]).values
                                                             ,item_features=item_features 
                                                             , user_features=user_features
                                                              )
    
print(f'Mean Score: {np.mean(score_ls):0.5f}')
print(f'Std Score: {np.std(score_ls):0.5f}')
print(f'Max Score: {np.max(score_ls):0.5f}')
print(f'Min Score: {np.min(score_ls):0.5f}')    

1 Fold Test MAPE: 0.7550
2 Fold Test MAPE: 0.7565
3 Fold Test MAPE: 0.7517
4 Fold Test MAPE: 0.7549
5 Fold Test MAPE: 0.7548

Mean Score: 0.75458
Std Score: 0.00158
Max Score: 0.75648
Min Score: 0.75166


In [ ]:
sample_submissions['res'] = sample_submissions.mean(axis=1)
preds = sample_submissions['res']
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()
submission['rating']= normalized_preds

In [ ]:
submission

,Id,rating
0,0,0.635997
1,1,0.610852
2,2,0.607010
3,3,0.452351
4,4,0.506215
...,...,...
285960,285960,0.679087
285961,285961,0.404251
285962,285962,0.509674
285963,285963,0.584099


In [ ]:
VERSION = 23
submission.to_csv(DIR_TEST+ f'submission_v{VERSION}.csv', index=False)

In [ ]:
submission

,Id,rating
0,0,0.650563
1,1,0.615751
2,2,0.607787
3,3,0.452588
4,4,0.524707
...,...,...
285960,285960,0.683191
285961,285961,0.426928
285962,285962,0.507427
285963,285963,0.589488


Отличный результат на тесте, но на каггле дает только 0.77494